# Module 4 Exercise 2 - Grid Search for the best CNN

## **NOTE: You need to use the Tensorflow CPU container for this exercise**

## Overview
In this exercise, you will perform a grid search using the model defined in lab-04-03.  The model and associated code are provided.  You must write the code to load the data and do the grid search.

## File Format

The data are separated into two directories, one for postive samples (a tumor exists), and one for negative. In each directory there are a number of jpeg files of 32x32 pixels.  These are monochromatic images.

The directories are:
```
negative images: ../resources/cnn-images/negative_images/*.jpg
positive images: ../resources/cnn-images/positive_images/*.jpg
```

## Required Output
The output required for this exercise is the grid search that you will perform.  In your grid search, you must vary three parameters for model training.  These can be the epoch, batch size, and the learning rate. 

hint: look at the [Keras documentation for model.compile](https://keras.io/api/models/model_training_apis/) to see how to specify a constant learning rate
        
## Grading
This exercise is graded on the contents of your notebook only, as there is not a right or wrong answer to training a Convolutional Neural Network.  

You will receive a score of 25 if you have performed a grid search with 3 parameters that change value (at least two values for each).  You will receive 20 points if you run a grid search using only two parameters that change value, and 15 points for using only one parameter that changes value.

If your notebook doesn't run, you will receive 12 points.

If you don't supply any code to do a grid search, then you will receive a score of zero.


In [1]:
import sys
!{sys.executable} -m pip install keras==2.3.1
!{sys.executable} -m pip install --upgrade "numpy>=1.2"

import os
import numpy as np
import math
import glob
from PIL import Image
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from keras.optimizers import Adadelta, SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 378kB 1.8MB/s eta 0:00:01
     |████████████████████████████████| 15.7MB 3.3MB/s eta 0:00:01
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4


Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

## Create a function to build the CNN model

Use the first model built in the lab as a base.  Modify it as needed to allow for grid searching with a specified learning rate.

In [2]:
# your code here

def create_model(img_channels, img_rows, img_cols):
    # img_channels: how many color channels
    # img_rows and img_cols: the size of the image
    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = 3, padding='same', input_shape=(img_channels, img_rows, img_cols)))
    model.add(Activation('relu'))

    model.add(Conv2D(16, kernel_size = 5, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size = 2, data_format='channels_first'))

    model.add(Conv2D(16, kernel_size = 3, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, kernel_size = 5, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Conv2D(64, kernel_size = 3, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size = 2, data_format='channels_first'))

    model.add(Conv2D(128, kernel_size = 3, padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size = 2, data_format='channels_first'))
    
    model.add(Flatten())
    model.add(Dense(128, kernel_initializer="he_normal"))
    model.add(Activation('relu'))

    model.add(Dropout(0.5)) 
    model.add(Dense(32, kernel_initializer="he_normal"))
    model.add(Activation('relu'))

    model.add(Dropout(0.5)) 
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    # learning rate optimizer
    optimizer = Adadelta(lr=0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [19]:
img_rows=32
img_cols=32
img_channels=1

model = create_model(img_channels, img_rows, img_cols)

In [4]:
def read_image_data(filename):
    # read grayscale image data to an 2d numpy array
    image = Image.open(filename)
    image = image.getdata()
    image = np.array(image)
    return image.reshape(-1)


def image_dir_to_array(dir):
    data = [read_image_data(image) for image in glob.glob(os.path.join(dir, '*.jpg'))]
    return np.array(data)


def load_data(negative_images_path, positive_images_path):
    negatives = image_dir_to_array(negative_images_path)
    positives = image_dir_to_array(positive_images_path)
    
    X=np.vstack((negatives, positives))
    X=X.astype(np.float) / 255 # reduce colordepth normalize the image grayscale values from 0..1
    y=np.concatenate((np.zeros(len(negatives)), np.ones(len(positives))))
    
    print ('shape of X', np.shape(X)) 
    print ('scale of X', np.min(X), np.max(X))
    print ('shape of y', np.shape(y)) 
    
    return X, y

def reshape_X(X, img_channels, img_rows, img_cols):
    # reshape the data to the 4 dimensional format required by the CNN
    # the resulting shape will be (num_samples, img_channels (1 for grayscale images), img_rows, img_cols)
    return X.reshape(-1, img_channels, img_rows, img_cols)

## Load the data

In [5]:
# your code here

negative_images_path = '../resources/cnn-images/negative_images/'
positive_images_path = '../resources/cnn-images/positive_images/'

X, y = load_data(negative_images_path, positive_images_path)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = reshape_X(X_train, img_channels, img_rows, img_cols)
X_test = reshape_X(X_test, img_channels, img_rows, img_cols)

shape of X (8710, 1024)
scale of X 0.0 1.0
shape of y (8710,)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


## Create and execute the grid search

Create the parameter grid, a wrapped model, and execute the grid search.

Output the best model's accuracy and it's associated parameters.

Warning: Do not use excessively large number of epochs, or you will be waiting for a long time for the grid search to complete.

In [21]:
# your code here

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, verbose=2)

param_grid = {
    'img_channels': [img_channels],
    'img_rows': [img_rows],
    'img_cols': [img_cols],
    'epochs': [2, 3],
    'batch_size': [32, 64],
    'class_weight': ['balanced', None]
}

X = reshape_X(X, img_channels, img_rows, img_cols) # we have to reshape our input to match what the CNN expects

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv = 3, verbose=2)
grid_result = grid.fit(X, y, shuffle=True)

print("Best model: %f with parameters %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Epoch 1/3
 - 5s - loss: 2.4293 - accuracy: 0.3799
Epoch 2/3
 - 4s - loss: 2.4353 - accuracy: 0.3853
Epoch 3/3
 - 4s - loss: 2.4673 - accuracy: 0.3788
Best model: 1.000000 with parameters {'batch_size': 64, 'class_weight': None, 'epochs': 3, 'img_channels': 1, 'img_cols': 32, 'img_rows': 32}
